In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
listings = pd.read_csv("/content/cleaned_listing_v3")

# Data Preprocessing/Validity Check

In [ ]:
listings = listings.assign(
    listing_id = lambda x: x.listing_id.astype("object")
)

In [ ]:
calendar = pd.read_csv("/content/cleaned_calendar_v3")

/tmp/ipython-input-1292462284.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  calendar = pd.read_csv("/content/cleaned_calendar_v3")


In [ ]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225881 entries, 0 to 1225880
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   listing_id      1225881 non-null  int64  
 1   date            1225880 non-null  object 
 2   available       1225880 non-null  object 
 3   minimum_nights  1225880 non-null  float64
 4   maximum_nights  1225880 non-null  float64
 5   price           1225880 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 56.1+ MB


In [ ]:
calendar = calendar.assign(
    listing_id = lambda x: x.listing_id.astype('object'),
    date = lambda x: pd.to_datetime(x.date)
)

In [ ]:
calendar.minimum_nights.isna().sum()

np.int64(1)

In [ ]:
calendar.maximum_nights.isna().sum()

np.int64(1)

In [ ]:
drop_empty_row = calendar.loc[(calendar.minimum_nights.isna()) | (calendar.maximum_nights.isna())].index

In [ ]:
calendar.shape[0]

1225881

In [ ]:
calendar.drop(drop_empty_row, axis=0, inplace=True)

In [ ]:
calendar = calendar.assign(
    minimum_nights = lambda x: x.minimum_nights.astype('int64'),
    maximum_nights = lambda x: x.maximum_nights.astype('int64')
)

In [ ]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225880 entries, 0 to 1225879
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   listing_id      1225880 non-null  object        
 1   date            1225880 non-null  datetime64[ns]
 2   available       1225880 non-null  object        
 3   minimum_nights  1225880 non-null  int64         
 4   maximum_nights  1225880 non-null  int64         
 5   price           1225880 non-null  float64       
 6   booked          1225880 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 65.5+ MB


In [ ]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4741 entries, 0 to 4740
Data columns (total 62 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   listing_id                                    4741 non-null   object 
 1   name                                          4741 non-null   object 
 2   description                                   4680 non-null   object 
 3   neighborhood_overview                         2977 non-null   object 
 4   host_name                                     4739 non-null   object 
 5   host_since                                    4739 non-null   object 
 6   host_about                                    3044 non-null   object 
 7   host_response_time                            4461 non-null   object 
 8   host_response_rate                            4461 non-null   object 
 9   host_acceptance_rate                          4552 non-null   o

In [ ]:
listings.to_csv('cleaned_listing_v4', index=False)

In [ ]:
calendar.to_csv('cleaned_calendar_v4', index=False)

In [ ]:
pd.set_option('display.max_columns', len(listings.columns))

# Creating variable that simulates DEMAND q(p)

In [ ]:
calendar.head()

,listing_id,date,available,minimum_nights,maximum_nights,price
0,3686,2025-03-13,True,1,365,67.0
1,3686,2025-03-14,True,31,365,67.0
2,3686,2025-03-15,True,31,365,67.0
3,3686,2025-03-16,True,31,365,67.0
4,3686,2025-03-17,True,31,365,67.0


In [ ]:
calendar = calendar.assign(
    booked = lambda x: np.where(x.available, 0, 1)
)

In [ ]:
calendar.head(3)

,listing_id,date,available,minimum_nights,maximum_nights,price,booked
0,3686,2025-03-13,True,1.0,365.0,67.0,0
1,3686,2025-03-14,True,31.0,365.0,67.0,0
2,3686,2025-03-15,True,31.0,365.0,67.0,0


In [ ]:
calendar.groupby('listing_id').booked.mean().head() # probability that the listing is BOOKED (demanded)

,booked
listing_id,
3686,0.000000
3943,0.043836
4197,0.035616
4529,0.509589
5589,0.567123


In [ ]:
calendar.groupby('listing_id').booked.mean().reset_index(name='occupancy_rate')

,listing_id,occupancy_rate
0,3686,0.000000
1,3943,0.043836
2,4197,0.035616
3,4529,0.509589
4,5589,0.567123
...,...,...
6176,1373918040709168206,0.219178
6177,1373919267253560643,0.736986
6178,1373919946920614774,0.049180
6179,1374061794031156604,0.863014


In [ ]:
occupancy = calendar.groupby('listing_id').booked.mean().reset_index(name='occupancy_rate')

In [ ]:
listings = listings.merge(occupancy, on="listing_id", how="left")

In [ ]:
listings.head()

,listing_id,name,description,neighborhood_overview,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,price,occupancy_rate
0,3686,Vita's Hideaway,IMPORTANT NOTES<br />* Carefully read and be s...,We love that our neighborhood is up and coming...,Vita,2008-11-26,"I am a literary scholar, teacher, poet, vegan ...",NaN,NaN,100%,False,1.0,4.0,"['email', 'phone', 'work_email']",True,True,Neighborhood highlights,Historic Anacostia,False,38.86339,-76.98889,Private room in home,Private room,1,1.0,1 shared bath,1.0,2.0,"[""Shampoo"", ""Smoke alarm"", ""Essentials"", ""Stov...",31,365,31.0,365.0,False,True,1,1,1,1,84,0,0,0,0,0,0.0,2010-11-01,2023-08-30,4.64,4.73,4.45,4.91,4.84,3.91,4.64,False,1,0,1,0,0.48,60.0,0.000000
1,3943,Historic Rowhouse Near Monuments,You will be staying in high ceiling bedroom w...,This rowhouse is centrally located in the hear...,Vasa,2008-12-12,I travel often and always try to immerse mysel...,within an hour,100%,95%,True,5.0,5.0,"['email', 'phone']",True,True,Neighborhood highlights,"Edgewood, Bloomingdale, Truxton Circle, Eckington",False,38.91195,-77.00456,Private room in townhouse,Private room,2,1.0,1 private bath,1.0,1.0,"[""Luggage dropoff allowed"", ""Window AC unit"", ...",1,1125,1.0,1125.0,False,True,14,44,74,349,534,38,1,278,36,228,14364.0,2009-05-10,2025-02-19,4.86,4.89,4.93,4.94,4.92,4.60,4.77,False,5,0,5,0,2.77,63.0,0.043836
2,4197,Capitol Hill Bedroom walk to Metro,This is the middle bedroom upstairs in a resto...,"Our area, the Eastern Market neighborhood of C...",Sandra,2008-12-12,I’m a California native who came to work for t...,within an hour,100%,100%,True,2.0,2.0,"['email', 'phone']",True,True,Neighborhood highlights,"Capitol Hill, Lincoln Park",False,38.88719,-76.99472,Private room in home,Private room,1,1.5,1.5 shared baths,1.0,1.0,"[""Luggage dropoff allowed"", ""Fire pit"", ""Shamp...",4,29,4.0,1125.0,False,True,21,47,77,352,64,6,0,281,5,48,6144.0,2009-05-14,2025-01-30,4.87,4.98,4.89,5.00,4.98,4.97,4.93,False,2,0,2,0,0.33,128.0,0.035616
3,4529,Bertina's House Part One,This is large private bedroom with plenty of...,Very quiet neighborhood and it is easy accessi...,Bertina,2008-12-30,I am a retried teacher. I enjoy walking and wa...,NaN,NaN,NaN,False,4.0,4.0,"['email', 'phone']",True,True,Neighborhood highlights,"Eastland Gardens, Kenilworth",False,38.90585,-76.94469,Private room in home,Private room,2,1.0,1 private bath,1.0,2.0,"[""Luggage dropoff allowed"", ""Shampoo"", ""Smoke ...",30,1125,30.0,1125.0,False,True,29,59,89,179,102,0,0,179,0,0,0.0,2009-08-08,2019-07-05,4.66,4.80,4.60,4.93,4.93,4.51,4.83,False,2,0,2,0,0.54,64.0,0.509589
4,7103,Lovely guest suite in a quiet but close-in nei...,"Private guest suite with cathedral ceiling, su...",NaN,Charlotte,2009-05-18,"My business is LUXbnb, we offer short-stay fur...",within an hour,100%,100%,True,30.0,106.0,"['email', 'phone', 'work_email']",True,True,NaN,"Spring Valley, Palisades, Wesley Heights, Foxh..."

In [ ]:
listings.occupancy_rate.describe().T

,occupancy_rate
count,4719.000000
mean,0.437666
std,0.313919
min,0.000000
25%,0.147945
50%,0.400000
75%,0.723288
max,1.000000


We now have an x-variable (price) and a proxy for demand q(p) (occupancy_rate)

# Attempting to plot the PRICE RESPONSE FUNCTION

In [ ]:
calendar.date.head()

,date
0,2025-03-13
1,2025-03-14
2,2025-03-15
3,2025-03-16
4,2025-03-17


In [ ]:
calendar.tail() # notice that these dates are in 2026

,listing_id,date,available,minimum_nights,maximum_nights,price,booked
2255553,8964989,2026-03-10,False,2,1125,99.0,1
2255554,8964989,2026-03-11,False,2,1125,99.0,1
2255555,8964989,2026-03-12,False,3,1125,99.0,1
2255556,8964989,2026-03-13,False,3,1125,99.0,1
2255557,8964989,2026-03-14,False,3,1125,99.0,1


**NOTE ON MAJOR ASSUMPTION**: The AVAILABLE column indicates whether the listing is open for booking on that date. Does not confirm whether or not it IS or HAS BEEN posted that day.

 - True == available for booking
 - False == not available for booking (it's either booked or currently blocked by host, at time of snapshot of data).

When AVAILABLE == False, we don't know if the listing was actually ever available, if the listing was removed because no one was booking (demand low), or some other reason.

When AVAILABLE == True, we don't know if the listing was just posted (no timestamp).

AVAILABLE == FALSE assumption may OVERESTIMATE DEMAND.
AVAILABLE == TRUE assumption may further blur the lines. Reflects SUPPLY, rather than DEMAND.

**NOTES ON THE DATA**
This data is published by Airbnb and projects listings for up to a year into the future.

The data was scrapped by Airbnb on March 13th. Therefore, dates AFTER March 13 2025 (into 2026) are simply FUTURE SCHEDULED PROJECTION. This tells us SUPPLY, rather than REALIZED DEMAND.

We can't predict nor infer HOW customers will respond to these future prices yet - because they simply haven't been realized to the market yet.

**ESTIMATING THE PRICE-RESPONSE FUNCTION AND 2026 LISTINGS**
We need OBSERVED DEMAND OUTCOMES.
Future listings (Feb-March 2026) don't serve as proxies for real demand behavior - they're placeholders.
Including them would:
 - Make demand look like there is none more frequently than reality.
 - Bias elasticity estimates (demand will seem unresponsive to changing price)
 - Bias revenue estimates.

**Defining the relevant data**
Want listings that reflect recent and realized demand.

DECISION:
 - Drop all 2026 listings (consider setting aside for future forecasting tests).
 - Keep only postings made before or shortly after March 13 (later listings likely represent demand planning).

In [ ]:
calendar.head()

,listing_id,date,available,minimum_nights,maximum_nights,price,booked
0,3686,2025-03-13,True,1,365,67.0,0
1,3686,2025-03-14,True,31,365,67.0,0
2,3686,2025-03-15,True,31,365,67.0,0
3,3686,2025-03-16,True,31,365,67.0,0
4,3686,2025-03-17,True,31,365,67.0,0


## Exploring data to determine practical filtering procedure

**GOAL**: Filter out and set aside anything that is FUTURE-DATE, EMPTY, or INACTIVE

### Removing all 2026 listings

Intend to set them aside. Potentially later use to either:
 - Forecast testing or LLM scenario generation. Potentially "How would fitted model predict 2026 occupancy given price?" or "Given listing's historical elasticity (March 2025, for example), what's the expected occupancy in March 2026?"

In [ ]:
_2026 = pd.to_datetime('2026-01-01')

In [ ]:
_2026_calendar = calendar.loc[lambda x: pd.to_datetime(x.date) >= _2026]

In [ ]:
_2026_calendar.head(1)

,listing_id,date,available,minimum_nights,maximum_nights,price,booked
294,3686,2026-01-01,True,31,365,67.0,0


In [ ]:
calendar.head(3)

,listing_id,date,available,minimum_nights,maximum_nights,price,booked
0,3686,2025-03-13,True,1,365,67.0,0
1,3686,2025-03-14,True,31,365,67.0,0
2,3686,2025-03-15,True,31,365,67.0,0


**ISSUE**
Plan was to filter out and set aside listings that don't represent DEMAND, any SUPPLY related postings (e.g., 2026 listings). However, the same *listing_id's* occassionally appear in both 2025 and 2026 entries because the ID represents a single Airbnb listing that continues to exist.
 - Each listting actually has many rows because each date represents a date Airbnb PUBLISHED that posting.

Filtering out using LISTING_ID could potentially remove the ENTIRE LISTING

In [ ]:
listings.head().iloc[:,-1]

,occupancy_rate
0,0.000000
1,0.043836
2,0.035616
3,0.509589
4,0.150685


In [ ]:
# dropping occupancy_rate because not accurate representation for demand yet
listings.drop(columns=["occupancy_rate"], inplace=True)

In [ ]:
# dropping derived features from CALENDAR because not accurate yet
calendar.drop(columns=["booked"], inplace=True)

*QUESTION*

How much data would we have to use for fitting model if we only include instances UP to the scrap date?

In [ ]:
scrap_date = pd.to_datetime('2025-03-13')

In [ ]:
calendar.sort_values(by="date")

,listing_id,date,available,minimum_nights,maximum_nights,price
0,3686,2025-03-13,True,1,365,67.0
1428502,1010705782683167311,2025-03-13,False,1,1125,158.0
1428137,1010653751695329863,2025-03-13,False,3,7,130.0
363965,23148309,2025-03-13,False,2,1125,115.0
1427772,1010486489906800727,2025-03-13,False,183,1125,240.0
...,...,...,...,...,...,...
141647,10311634,2026-03-14,False,3,1125,180.0
1837736,1224731147854068394,2026-03-14,False,2,28,229.0
1497738,1049236007638467900,2026-03-14,False,1,1125,80.0
805312,50189968,2026-03-14,True,1,1125,129.0


In [ ]:
calendar.loc[lambda x:
             pd.to_datetime(x.date) <= scrap_date].sort_values(by="date")

# not sure if enough for training

,listing_id,date,available,minimum_nights,maximum_nights,price
0,3686,2025-03-13,True,1,365,67.0
1498104,1049371086899899688,2025-03-13,False,1,29,85.0
1496279,1048588275488319161,2025-03-13,False,1,365,165.0
1495914,1047916172762233921,2025-03-13,False,1,1125,125.0
1495549,1047230201172107759,2025-03-13,False,2,1125,577.0
...,...,...,...,...,...,...
726461,47451426,2025-03-13,False,4,1125,220.0
726096,47443903,2025-03-13,False,14,730,35.0
725731,47431222,2025-03-13,False,1,1125,70.0
729746,47560691,2025-03-13,False,1,29,77.0


In [ ]:
potential_buffer_date = pd.to_datetime("2025-06-01")
# potentially keep if these postings were available to public at time of scrapping

In [ ]:
calendar.loc[lambda x:
              pd.to_datetime(x.date) <= potential_buffer_date
].shape[0] # 499014 listings made BEFORE (or on) June 01 2025

271141

In [ ]:
relevant_calendar = calendar.loc[lambda x:
              pd.to_datetime(x.date) <= potential_buffer_date
] # candidate instances to derive price response function

In [ ]:
relevant_calendar.shape

(271141, 7)

## Adding BOOKED feature (proxy for demand) to postings more than likely representative of DEMAND

In [ ]:
relevant_calendar.head()

,listing_id,date,available,minimum_nights,maximum_nights,price
0,3686,2025-03-13,True,1,365,67.0
1,3686,2025-03-14,True,31,365,67.0
2,3686,2025-03-15,True,31,365,67.0
3,3686,2025-03-16,True,31,365,67.0
4,3686,2025-03-17,True,31,365,67.0


In [ ]:
relevant_calendar = relevant_calendar.assign(
    booked = lambda x: np.where(x.available, 0, 1) # Booked=1 if Available=False
)

In [ ]:
relevant_calendar

,listing_id,date,available,minimum_nights,maximum_nights,price,booked
0,3686,2025-03-13,True,1,365,67.0,0
1,3686,2025-03-14,True,31,365,67.0,0
2,3686,2025-03-15,True,31,365,67.0,0
3,3686,2025-03-16,True,31,365,67.0,0
4,3686,2025-03-17,True,31,365,67.0,0
...,...,...,...,...,...,...,...
1225591,861857172153830345,2025-05-28,True,2,10,5000.0,0
1225592,861857172153830345,2025-05-29,False,2,10,5000.0,1
1225593,861857172153830345,2025-05-30,False,3,10,5000.0,1
1225594,861857172153830345,2025-05-31,False,3,10,5000.0,1


## INSIGHT:
Observe that (above), in using the calendar data to construct a proxy for realized demand, we must be careful not to double-count the same booking across consecutive days.

The same five rows above represent a SINGLE BOOKING, not five separate units of booking/demand. If we aren't careful, we could overstand demand rather easily here.

NOTE: It is positive to see that listings towards the end of May were indeed viable and available listings, as the listing (830345) couldn't have been booked if it wasn't available.

# Attempting to develop a strategy to identify "Repeat" Bookings

Idea - count ONLY unique bookings (the first instance/day the listing transitions from available to booked).

In [ ]:
calendar

,listing_id,date,available,minimum_nights,maximum_nights,price,booked
0,3686,2025-03-13,True,1,365,67.0,0
1,3686,2025-03-14,True,31,365,67.0,0
2,3686,2025-03-15,True,31,365,67.0,0
3,3686,2025-03-16,True,31,365,67.0,0
4,3686,2025-03-17,True,31,365,67.0,0
...,...,...,...,...,...,...,...
1225875,861857172153830345,2026-03-08,True,5,10,5000.0,0
1225876,861857172153830345,2026-03-09,True,5,10,5000.0,0
1225877,861857172153830345,2026-03-10,True,5,10,5000.0,0
1225878,861857172153830345,2026-03-11,True,5,10,5000.0,0


In [ ]:
pd.Series(calendar.sort_values(['listing_id', 'date']).index).equals(pd.Series(calendar.index.values))
# need ensure that each listing is sorted in proper sequential manner


False

In [ ]:
calendar = calendar.sort_values(['listing_id', 'date'])

In [ ]:
calendar_copy = calendar.assign(
    new_booking = (calendar.available.shift(1) == True # return all rows in which the row prior to it has a True value in available ..
 ) & (calendar.available == False # ... and the row in question has available = FALSE
                                         ) & (
                                             calendar.listing_id == calendar.listing_id.shift(1)
                                         ) # ... and the rows listing_id equals the listing_id of the row before it
)

In [ ]:
calendar_copy.loc[
    (calendar_copy.new_booking==True) & (calendar_copy.new_booking.shift(1) == False)
]

,listing_id,date,available,minimum_nights,maximum_nights,price,booked,new_booking
373,3943,2025-03-20,False,1,1125,55.0,1,True
387,3943,2025-04-03,False,1,1125,55.0,1,True
755,4197,2025-04-06,False,4,1125,83.0,1,True
772,4197,2025-04-23,False,4,1125,83.0,1,True
1276,4529,2025-09-09,False,30,1125,55.0,1,True
...,...,...,...,...,...,...,...,...
1225613,861857172153830345,2025-06-19,False,2,10,5000.0,1,True
1225627,861857172153830345,2025-07-03,False,2,10,5000.0,1,True
1225633,861857172153830345,2025-07-09,False,2,10,5000.0,1,True
1225641,861857172153830345,2025-07-17,False,2,10,5000.0,1,True


In [ ]:
calendar_copy.loc[lambda x: x.listing_id == 4197].head(25)

,listing_id,date,available,minimum_nights,maximum_nights,price,booked,new_booking
731,4197,2025-03-13,False,4,1125,83.0,1,False
732,4197,2025-03-14,False,4,1125,83.0,1,False
733,4197,2025-03-15,False,4,1125,83.0,1,False
734,4197,2025-03-16,False,4,1125,83.0,1,False
735,4197,2025-03-17,True,4,1125,83.0,0,False
736,4197,2025-03-18,True,4,1125,83.0,0,False
737,4197,2025-03-19,True,4,1125,83.0,0,False
738,4197,2025-03-20,True,4,1125,83.0,0,False
739,4197,2025-03-21,True,4,1125,83.0,0,False
740,4197,2025-03-22,True,4,1125,83.0,0,False


# INSIGHT FROM ABOVE:
Notice that, because the dataset snapshot begins at the onset of March 13th, the listings, such as the one above for example, that were unavailable (due in all likelihood to being booked) dont register as a NEW BOOKING.

This is because the observation window (Airbnb listings in Washington DC made on or after March 13th 2025) started AFTER listings were already available and welcominging guests (demand was being realized).

If we don't handle this properly, we may:
 - UNDER-COUNT BOOKINGS early on in the data (all bookings that began and were made before March 13th)
 - bias our demand proxy downward, especially at the onset of our modeling.
 - distort the pricing/demand relationships hidden in the data.

In [ ]:
calendar_copy

,listing_id,date,available,minimum_nights,maximum_nights,price,booked,new_booking,prev_availability
0,3686,2025-03-13,True,1,365,67.0,0,False,NaN
1,3686,2025-03-14,True,31,365,67.0,0,False,True
2,3686,2025-03-15,True,31,365,67.0,0,False,True
3,3686,2025-03-16,True,31,365,67.0,0,False,True
4,3686,2025-03-17,True,31,365,67.0,0,False,True
...,...,...,...,...,...,...,...,...,...
1225875,861857172153830345,2026-03-08,True,5,10,5000.0,0,False,True
1225876,861857172153830345,2026-03-09,True,5,10,5000.0,0,False,True
1225877,861857172153830345,2026-03-10,True,5,10,5000.0,0,False,True
1225878,861857172153830345,2026-03-11,True,5,10,5000.0,0,False,True


In [ ]:
g = calendar_copy.groupby('listing_id', sort=False)

In [ ]:
calendar_copy.groupby('listing_id', sort=False).available.shift(1)

,available
0,NaN
1,True
2,True
3,True
4,True
...,...
1225875,True
1225876,True
1225877,True
1225878,True


In [ ]:
calendar_copy = calendar_copy.assign(
    prev_availability = g.available.shift(1)
) # each rows PREV_AVAILABILITY column equals that listing's availability from the previous day

In [ ]:
calendar = calendar_copy.assign(
    new_booking = (calendar_copy.available==False) & (
        calendar_copy.prev_availability.fillna(True)
    )
) # NEW_BOOKING holds TRUE only if the listing had no previously posted availability recorded (initial posting occured outside of window)
# AND the instance must have just switched from AVAILABLE=TRUE to AVAILABLE=FALSE (demand realized)

/tmp/ipython-input-4162547285.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  calendar_copy.prev_availability.fillna(True)


In [ ]:
calendar.drop(columns=['prev_availability'], inplace=True)

In [ ]:
calendar.head() # new_booking==True means new booking started on that date (demand realized)

,listing_id,date,available,minimum_nights,maximum_nights,price,booked,new_booking
0,3686,2025-03-13,True,1,365,67.0,0,False
1,3686,2025-03-14,True,31,365,67.0,0,False
2,3686,2025-03-15,True,31,365,67.0,0,False
3,3686,2025-03-16,True,31,365,67.0,0,False
4,3686,2025-03-17,True,31,365,67.0,0,False


### Deriving new column to assess and account for continuous bookings

Idea here is that each initial TRUE instance in NEW_BOOKING will start a new BOOKING with a new unique booking_id. The booking_id doesn't change until a new_booking (demand) is realized for the property

In [ ]:
calendar = calendar.assign(
    booking_id = lambda x: x.groupby('listing_id').new_booking.cumsum()
) # gives every BOOKING a unique ID within the listing_id of the property

In [ ]:
calendar

,listing_id,date,available,minimum_nights,maximum_nights,price,booked,new_booking,booking_id
0,3686,2025-03-13,True,1,365,67.0,0,False,0
1,3686,2025-03-14,True,31,365,67.0,0,False,0
2,3686,2025-03-15,True,31,365,67.0,0,False,0
3,3686,2025-03-16,True,31,365,67.0,0,False,0
4,3686,2025-03-17,True,31,365,67.0,0,False,0
...,...,...,...,...,...,...,...,...,...
1225875,861857172153830345,2026-03-08,True,5,10,5000.0,0,False,14
1225876,861857172153830345,2026-03-09,True,5,10,5000.0,0,False,14
1225877,861857172153830345,2026-03-10,True,5,10,5000.0,0,False,14
1225878,861857172153830345,2026-03-11,True,5,10,5000.0,0,False,14


Above logic isn't as efficient because each booking_id should only describe instances in which the listing is ACTUALLY being occupied

In [ ]:
calendar = calendar.assign(
    booking_seq = lambda x: x.groupby('listing_id').new_booking.cumsum() # column to run count of booking starts within each listing
).drop(columns=['booking_id'])

In [ ]:
booked_condition = (calendar.available==False) & (calendar.booked==1)
# instance holds TRUE only if actually booked

In [ ]:
calendar = calendar.assign(
    booking_id = calendar.booking_seq.where(booked_condition) # booking label only kept on TRUE booked days; NA otherwise
)

In [ ]:
calendar.loc[lambda x: x.available==False]

,listing_id,date,available,minimum_nights,maximum_nights,price,booked,new_booking,booking_seq,booking_id
366,3943,2025-03-13,False,1,1125,55.0,1,True,1,1.0
367,3943,2025-03-14,False,1,1125,55.0,1,False,1,1.0
373,3943,2025-03-20,False,1,1125,55.0,1,True,2,2.0
374,3943,2025-03-21,False,1,1125,55.0,1,False,2,2.0
375,3943,2025-03-22,False,1,1125,55.0,1,False,2,2.0
...,...,...,...,...,...,...,...,...,...,...
1225643,861857172153830345,2025-07-19,False,4,10,5000.0,1,False,13,13.0
1225646,861857172153830345,2025-07-22,False,2,10,5000.0,1,True,14,14.0
1225647,861857172153830345,2025-07-23,False,2,10,5000.0,1,False,14,14.0
1225648,861857172153830345,2025-07-24,False,2,10,5000.0,1,False,14,14.0


### Attempting to make column that tracks length of stay of booking

In [ ]:
stay_length = (calendar.loc[booked_condition].groupby(
    ['listing_id', 'booking_id']
).date.size().rename('stay_length').reset_index())

In [ ]:
stay_length.head()

,listing_id,booking_id,stay_length
0,3943,1.0,2
1,3943,2.0,11
2,3943,3.0,3
3,4197,1.0,4
4,4197,2.0,5


## Attempting to capture start and end dates of bookings to explicity define duration time stamped interval for each realized booking.

Idea is that this allowes for time-based demand analysis (seasonality/weekday effects (i.e., on what day doo most check-ins occur)

In [ ]:
start_end_dates = (calendar.loc[booked_condition].groupby(
    ['listing_id', 'booking_id']
).date.agg(start_date='min', end_date='max').reset_index())

In [ ]:
start_end_dates.head()

,listing_id,booking_id,start_date,end_date
0,3943,1.0,2025-03-13,2025-03-14
1,3943,2.0,2025-03-20,2025-03-30
2,3943,3.0,2025-04-03,2025-04-05
3,4197,1.0,2025-03-13,2025-03-16
4,4197,2.0,2025-04-06,2025-04-10


### Merging back derived features to calendar data

In [ ]:
stay_length_copy = stay_length[['listing_id', 'booking_id', 'stay_length']].copy()

In [ ]:
start_end_dates_copy = start_end_dates[
    ['listing_id', 'booking_id', 'start_date', 'end_date']
].copy()

In [ ]:
booking_info = stay_length_copy.merge(
    start_end_dates, on=['listing_id', 'booking_id'], how='left',
    validate='one_to_one'
)

### Making sure datatypes match on the dimensions being joined

In [ ]:
booking_info = booking_info.assign(
    listing_id = lambda x: x.listing_id.astype('object')
)

In [ ]:
booking_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19700 entries, 0 to 19699
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   19700 non-null  object        
 1   booking_id   19700 non-null  float64       
 2   stay_length  19700 non-null  int64         
 3   start_date   19700 non-null  datetime64[ns]
 4   end_date     19700 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(1)
memory usage: 769.7+ KB


In [ ]:
booking_info.head()

,listing_id,booking_id,stay_length,start_date,end_date
0,3943,1.0,2,2025-03-13,2025-03-14
1,3943,2.0,11,2025-03-20,2025-03-30
2,3943,3.0,3,2025-04-03,2025-04-05
3,4197,1.0,4,2025-03-13,2025-03-16
4,4197,2.0,5,2025-04-06,2025-04-10


In [ ]:
booking_info = booking_info.assign(
    stay_length = lambda x: x.stay_length.astype('float64')
)

In [ ]:
booking_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19700 entries, 0 to 19699
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   19700 non-null  object        
 1   booking_id   19700 non-null  float64       
 2   stay_length  19700 non-null  float64       
 3   start_date   19700 non-null  datetime64[ns]
 4   end_date     19700 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 769.7+ KB


In [ ]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225880 entries, 0 to 1225879
Data columns (total 13 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   listing_id      1225880 non-null  object        
 1   date            1225880 non-null  datetime64[ns]
 2   available       1225880 non-null  object        
 3   minimum_nights  1225880 non-null  int64         
 4   maximum_nights  1225880 non-null  int64         
 5   price           1225880 non-null  float64       
 6   booked          1225880 non-null  int64         
 7   new_booking     1225880 non-null  bool          
 8   booking_seq     1225880 non-null  int64         
 9   booking_id      600380 non-null   float64       
 10  stay_length     600380 non-null   float64       
 11  start_date      600380 non-null   datetime64[ns]
 12  end_date        600380 non-null   datetime64[ns]
dtypes: bool(1), datetime64[ns](3), float64(3), int64(4), object(2)
memory us

In [ ]:
booking_info = booking_info.assign(
    stay_length = lambda x: x.stay_length.astype('Int64')
)

In [ ]:
calendar = calendar.assign(
    stay_length = lambda x: x.stay_length.astype('Int64') # creates a nullable column that holds integers but accepts NULL values
)

In [ ]:
calendar = calendar.drop(columns=['stay_length', 'start_date', 'end_date']).merge(
    booking_info, on=['listing_id', 'booking_id'], how='left',
    validate='many_to_one'
)

In [ ]:
calendar.loc[lambda x: x.available==False]

,listing_id,date,available,minimum_nights,maximum_nights,price,booked,new_booking,booking_seq,booking_id,stay_length,start_date,end_date
366,3943,2025-03-13,False,1,1125,55.0,1,True,1,1.0,2,2025-03-13,2025-03-14
367,3943,2025-03-14,False,1,1125,55.0,1,False,1,1.0,2,2025-03-13,2025-03-14
373,3943,2025-03-20,False,1,1125,55.0,1,True,2,2.0,11,2025-03-20,2025-03-30
374,3943,2025-03-21,False,1,1125,55.0,1,False,2,2.0,11,2025-03-20,2025-03-30
375,3943,2025-03-22,False,1,1125,55.0,1,False,2,2.0,11,2025-03-20,2025-03-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225643,861857172153830345,2025-07-19,False,4,10,5000.0,1,False,13,13.0,3,2025-07-17,2025-07-19
1225646,861857172153830345,2025-07-22,False,2,10,5000.0,1,True,14,14.0,4,2025-07-22,2025-07-25
1225647,861857172153830345,2025-07-23,False,2,10,5000.0,1,False,14,14.0,4,2025-07-22,2025-07-25
1225648,861857172153830345,2025-07-24,False,2,10,5000.0,1,False,14,14.0,4,2025-07-22,2025-07-25


Because it is evident that within the CALENDAR level data, a single booking occupiers multiple rows, we are going to make a separate booking_table, in which each row represent a SINGULAR continuous stay).

May prove useful down the line.

In [ ]:
booking_level = calendar.loc[calendar.new_booking].rename(
    columns={'date':'booking_start_date'}
)[['listing_id', 'booking_id', 'booking_start_date', 'stay_length',
   'start_date', 'end_date']]

In [ ]:
booking_level.head(30)

,listing_id,booking_id,booking_start_date,stay_length,start_date,end_date
366,3943,1.0,2025-03-13,2.0,2025-03-13,2025-03-14
373,3943,2.0,2025-03-20,11.0,2025-03-20,2025-03-30
387,3943,3.0,2025-04-03,3.0,2025-04-03,2025-04-05
731,4197,1.0,2025-03-13,4.0,2025-03-13,2025-03-16
755,4197,2.0,2025-04-06,5.0,2025-04-06,2025-04-10
772,4197,3.0,2025-04-23,4.0,2025-04-23,2025-04-26
1096,4529,1.0,2025-03-13,1.0,2025-03-13,2025-03-13
1276,4529,2.0,2025-09-09,185.0,2025-09-09,2026-03-12
1461,5589,1.0,2025-03-13,7.0,2025-03-13,2025-03-19
1495,5589,2.0,2025-04-16,15.0,2025-04-16,2025-04-30


In [ ]:
calendar_level = calendar.copy()

In [ ]:
calendar_level.head()

,listing_id,date,available,minimum_nights,maximum_nights,price,booked,new_booking,booking_seq,booking_id,stay_length,start_date,end_date
0,3686,2025-03-13,True,1,365,67.0,0,False,0,NaN,<NA>,NaT,NaT
1,3686,2025-03-14,True,31,365,67.0,0,False,0,NaN,<NA>,NaT,NaT
2,3686,2025-03-15,True,31,365,67.0,0,False,0,NaN,<NA>,NaT,NaT
3,3686,2025-03-16,True,31,365,67.0,0,False,0,NaN,<NA>,NaT,NaT
4,3686,2025-03-17,True,31,365,67.0,0,False,0,NaN,<NA>,NaT,NaT


In [ ]:
calendar_level.to_csv('calendar_level_csv', index=False)

In [ ]:
booking_level.to_csv('booking_level_csv', index=False)

## Can now begin moving onto next phase - modeling the underlying Price Response function, etc.